# COGS 108 - Final Project 

## Permissions

Place an `X` in the appropriate bracket below to specify if you would like your group's project to be made available to the public. (Note that PIDs will be scraped from the public submission, but student names will be included.)

* [  ] YES - make available
* [  ] NO - keep private

# Overview

*Fill in your overview here*

# Names

- Toland (Venti)
- Chongyun (Daiki)
- GO
- cw

# Group Members IDs

- A15747306(Kyle)
- A14136586
- A######## (Gary)
- A######## (Cary)

# Research Question

How does a games genre, merchandise, critic score, online/total players count, player base affect a games popularity and sales?

## Background and Prior Work

This question is interesting because:
We will be able to see the current trends in gaming.
When a game is released by a company, the critic score, the company its made by, and the genre can predict how well the game does in the market. These all influence the popularity of a game. So according to these variables, we want to predict how popular a game is and if it is worth playing. Recently some popular games such as among us and Genshin impact have skyrocketed in popularity and sales, so we want to investigate what exactly makes games like this popular so fast.
References (include links):
- 1)https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.549.6080&rep=rep1&type=pdf
- 2)

# Hypothesis


According to the paper above, adventure games and strategy games are the highest rated genres due to their storylines, graphics, and stimulating gameplay. We expect games that fit this category to have the highest sales, ratings, and playerbase.

# Dataset(s)

*Fill in your dataset information here*

(Copy this information for each dataset)
- Dataset Name: Metacritic Games Stats 2011-2019
- Link to the dataset: https://www.kaggle.com/skateddu/metacritic-games-stats-20112019?select=metacritic_games.csv
- Number of observations: 4018

1-2 sentences describing each dataset. 

If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.

*Fill in your dataset information here*

(Copy this information for each dataset)
- Dataset Name: Video Game Sales
- Link to the dataset: https://www.kaggle.com/ashaheedq/video-games-sales-2019?select=vgsales-12-4-2019-short.csv
- Number of observations: 37102

1-2 sentences describing each dataset. 

If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.

*Fill in your dataset information here*

(Copy this information for each dataset)
- Dataset Name: IGN Games of 20 Years
- Link to the dataset: https://raw.githubusercontent.com/john7obed/ign_games_of_20_years/master/ign.csv
- Number of observations: 18624

1-2 sentences describing each dataset. 

If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.

# Setup

Let's begin by importing some packages for analysis.

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

import patsy
import statsmodels.api as sm
import scipy.stats as stats
from scipy.stats import ttest_ind, chisquare, normaltest

In [2]:
# Configure libraries
# The seaborn library makes plots look nicer
sns.set()
sns.set_context('talk')

# Don't display too many rows/cols of DataFrames
pd.options.display.max_rows = 7
pd.options.display.max_columns = 8

# Round decimals when displaying DataFrames
pd.set_option('precision', 2)

Let's also import the 3 datasets listed for the analysis.

In [3]:
metacritic = pd.read_csv('metacritic_games.csv')
ign = pd.read_csv('ign.csv', index_col='Unnamed: 0')
vgsales = pd.read_csv('vgsales.csv')

To make sure that our data is imported properly:

In [4]:
metacritic.head()

,game,platform,developer,genre,...,neutral_users,negative_users,metascore,user_score
0,Portal 2,PC,Valve Software,Action,...,107,19,95,90
1,The Elder Scrolls V: Skyrim,PC,Bethesda Game Studios,Role-Playing,...,322,451,94,82
2,The Legend of Zelda: Ocarina of Time 3D,3DS,GREZZO,Miscellaneous,...,20,5,94,90
3,Batman: Arkham City,PC,Rocksteady Studios,Action Adventure,...,34,27,91,87
4,Super Mario 3D Land,3DS,Nintendo,Action,...,39,11,90,84


In [5]:
ign.head()

,score_phrase,title,url,platform,...,editors_choice,release_year,release_month,release_day
0,Amazing,LittleBigPlanet PS Vita,/games/littlebigplanet-vita/vita-98907,PlayStation Vita,...,Y,2012,9,12
1,Amazing,LittleBigPlanet PS Vita -- Marvel Super Hero E...,/games/littlebigplanet-ps-vita-marvel-super-he...,PlayStation Vita,...,Y,2012,9,12
2,Great,Splice: Tree of Life,/games/splice/ipad-141070,iPad,...,N,2012,9,12
3,Great,NHL 13,/games/nhl-13/xbox-360-128182,Xbox 360,...,N,2012,9,11
4,Great,NHL 13,/games/nhl-13/ps3-128181,PlayStation 3,...,N,2012,9,11


In [6]:
vgsales.head()

,Rank,Name,Platform,Year,...,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,...,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,...,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,...,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,...,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,...,8.89,10.22,1.00,31.37


# Data Cleaning

Describe your data cleaning steps here.

In [7]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION

We would like to merge the three DataFrames into one. First, we take a look at the columns of each DataFrame.

In [8]:
metacritic.columns

Index(['game', 'platform', 'developer', 'genre', 'number_players', 'rating',
       'release_date', 'positive_critics', 'neutral_critics',
       'negative_critics', 'positive_users', 'neutral_users', 'negative_users',
       'metascore', 'user_score'],
      dtype='object')

In [9]:
ign.columns

Index(['score_phrase', 'title', 'url', 'platform', 'score', 'genre',
       'editors_choice', 'release_year', 'release_month', 'release_day'],
      dtype='object')

In [10]:
vgsales.columns

Index(['Rank', 'Name', 'Platform', 'Year', 'Genre', 'Publisher', 'NA_Sales',
       'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales'],
      dtype='object')

In the datasets, same things are labelled differently, such as name of a game. Let's standardize the column labels of each entry before we do any data manipulation.

In [17]:
# vgsales column names should be all lowercase
vgsales = vgsales.rename(columns=str.lower)

# misc renames
metacritic = metacritic.rename(columns={'game': 'name'})
ign = ign.rename(columns={'title':'name'})
vgsales = vgsales.rename(columns={'year':'release_year', 'na_sales' : 'na_sales_million', 'eu_sales': 'eu_sales_million', 'jp_sales': 'jp_sales_million', 'other_sales': 'other_sales_million', 'global_sales': 'global_sales_million'})

Index(['rank', 'name', 'platform', 'release_year', 'genre', 'publisher',
       'na_sales_million', 'eu_sales_million', 'jp_sales_million',
       'other_sales_million', 'global_sales_million'],
      dtype='object')

There are some columns that are for sure irrelevant to the analysis, such as the URL path of a game review on IGN.

In [12]:
ign = ign.drop(columns='url')

In all datasets, although certain games may have same titles, they are treated as different observations because they may be in different platforms. Let's standardize our titles this way as well.

In [13]:
vgsales['platform'].unique()

array(['Wii', 'NES', 'GB', 'DS', 'X360', 'PS3', 'PS2', 'SNES', 'GBA',
       '3DS', 'PS4', 'N64', 'PS', 'XB', 'PC', '2600', 'PSP', 'XOne', 'GC',
       'WiiU', 'GEN', 'DC', 'PSV', 'SAT', 'SCD', 'WS', 'NG', 'TG16',
       '3DO', 'GG', 'PCFX'], dtype=object)

In [14]:
ign['platform'].unique()

array(['PlayStation Vita', 'iPad', 'Xbox 360', 'PlayStation 3',
       'Macintosh', 'PC', 'iPhone', 'Nintendo DS', 'Nintendo 3DS',
       'Android', 'Wii', 'PlayStation 4', 'Wii U', 'Linux',
       'PlayStation Portable', 'PlayStation', 'Nintendo 64', 'Saturn',
       'Lynx', 'Game Boy', 'Game Boy Color', 'NeoGeo Pocket Color',
       'Game.Com', 'Dreamcast', 'Dreamcast VMU', 'WonderSwan', 'Arcade',
       'Nintendo 64DD', 'PlayStation 2', 'WonderSwan Color',
       'Game Boy Advance', 'Xbox', 'GameCube', 'DVD / HD Video Game',
       'Wireless', 'Pocket PC', 'N-Gage', 'NES', 'iPod', 'Genesis',
       'TurboGrafx-16', 'Super NES', 'NeoGeo', 'Master System',
       'Atari 5200', 'TurboGrafx-CD', 'Atari 2600', 'Sega 32X', 'Vectrex',
       'Commodore 64/128', 'Sega CD', 'Nintendo DSi', 'Windows Phone',
       'Web Games', 'Xbox One', 'Windows Surface', 'Ouya',
       'New Nintendo 3DS', 'SteamOS'], dtype=object)

In [15]:
metacritic['platform'].unique()

array(['PC', '3DS', 'VITA', 'WIIU', 'PS4', 'XONE', 'Switch'], dtype=object)

In [25]:
#def standardize_platform(string):
vgsales[vgsales['name'].str.contains("Three Houses")]

,rank,name,platform,release_year,...,eu_sales_million,jp_sales_million,other_sales_million,global_sales_million
2056,2058,Breath of Fire III,PS,1997.0,...,0.19,0.46,0.07,1.00
3559,3561,Breath of Fire IV,PS,2000.0,...,0.08,0.34,0.04,0.56
5217,5219,Breath of Fire II,SNES,1994.0,...,0.00,0.36,0.00,0.36
7170,7172,Breath of Fire,GBA,2001.0,...,0.04,0.06,0.00,0.22
7758,7760,Breath of Fire II,GBA,2001.0,...,0.05,0.00,0.00,0.19
7789,7791,Breath of Fire: Dragon Quarter,PS2,2002.0,...,0.07,0.00,0.02,0.19
13636,13638,Breath of Fire III,PSP,2005.0,...,0.03,0.00,0.01,0.04


# Data Analysis & Results

Include cells that describe the steps in your data analysis.

In [ ]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION

# Ethics & Privacy

*Fill in your ethics & privacy discussion here*

# Conclusion & Discussion

*Fill in your discussion information here*

# Team Contributions

*Specify who in your group worked on which parts of the project.*